In [0]:
# Databricks notebook source
# ІМПОРТ ДАНИХ

olist_orders_df = spark.read.option("header", True).option("inferSchema", True)\
    .csv("/Volumes/workspace/default/olist_dataset/olist_orders_dataset.csv")

olist_orders_df.show(5, truncate=False)



+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

In [0]:
# COMMAND ----------

# ОЗНАЙОМЛЕННЯ ЗІ СТРУКТРОЮ ДАНИХ

from pyspark.sql.functions import col

# Розмір датасету
print("Кількість рядків:", olist_orders_df.count())
print("Кількість стовпців:", len(olist_orders_df.columns))

# Типи змінних та схема
print("Схема датасету:")
olist_orders_df.printSchema()


Кількість рядків: 99441
Кількість стовпців: 8
Схема датасету:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [0]:
# ПЕРВИННИЙ АНАЛІЗ ЯКОСТІ ДАНИХ

from pyspark.sql.functions import col, count, when, min, max, avg

# Підрахунок пропусків у кожній колонці
print("Кількість пропусків у кожній колонці:")
olist_orders_df.select([count(when(col(c).isNull(), c)).alias(c) for c in olist_orders_df.columns]).show()

# Виявлення дублікатів по order_id
print("Кількість дублікатів по order_id:")
duplicates_count = olist_orders_df.groupBy("order_id").count().filter("count > 1").count()
print(duplicates_count)


Кількість пропусків у кожній колонці:
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+

Кількість дублікатів по order_id:
0


In [0]:
# Перевірка категоріальних змінних (унікальні значення)
categorical_columns = ["order_status"]
for c in categorical_columns:
    print(f"Унікальні значення в колонці '{c}':")
    olist_orders_df.select(c).distinct().show(truncate=False)



Унікальні значення в колонці 'order_status':
+------------+
|order_status|
+------------+
|created     |
|delivered   |
|canceled    |
|invoiced    |
|shipped     |
|unavailable |
|processing  |
|approved    |
+------------+



In [0]:
# Перевірка часових полів на аномалії (мінімальні та максимальні дати)
date_columns = ["order_purchase_timestamp", "order_approved_at", 
                "order_delivered_carrier_date", "order_delivered_customer_date", 
                "order_estimated_delivery_date"]

for c in date_columns:
    print(f"Дати в колонці '{c}': min та max")
    olist_orders_df.select(min(c).alias("min_date"), max(c).alias("max_date")).show(truncate=False)

# Первинна перевірка числових полів (мінімум, максимум, середнє)
numeric_columns = [c[0] for c in olist_orders_df.dtypes if c[1] in ("int", "double")]
if numeric_columns:
    print("Аналіз числових колонок:")
    olist_orders_df.select(
        [min(c).alias(f"{c}_min") for c in numeric_columns] +
        [max(c).alias(f"{c}_max") for c in numeric_columns] +
        [avg(c).alias(f"{c}_avg") for c in numeric_columns]
    ).show(truncate=False)


Дати в колонці 'order_purchase_timestamp': min та max
+-------------------+-------------------+
|min_date           |max_date           |
+-------------------+-------------------+
|2016-09-04 21:15:19|2018-10-17 17:30:18|
+-------------------+-------------------+

Дати в колонці 'order_approved_at': min та max
+-------------------+-------------------+
|min_date           |max_date           |
+-------------------+-------------------+
|2016-09-15 12:16:38|2018-09-03 17:40:06|
+-------------------+-------------------+

Дати в колонці 'order_delivered_carrier_date': min та max
+-------------------+-------------------+
|min_date           |max_date           |
+-------------------+-------------------+
|2016-10-08 10:34:01|2018-09-11 19:48:28|
+-------------------+-------------------+

Дати в колонці 'order_delivered_customer_date': min та max
+-------------------+-------------------+
|min_date           |max_date           |
+-------------------+-------------------+
|2016-10-11 13:46:32|20

In [0]:
# ОЧИЩЕННЯ ДАНИХ

from pyspark.sql.functions import col, when, to_timestamp, lit, count

# Видалення дублікатів
olist_orders_cleaned = olist_orders_df.dropDuplicates(["order_id"])
print("Кількість рядків після видалення дублікатів:", olist_orders_cleaned.count())




Кількість рядків після видалення дублікатів: 99441


In [0]:
# Заповнення пропусків у часових колонках
# - якщо відсутня дата схвалення, то вважаємо, що це дата покупки
# - якщо немає дати передачі перевізнику, то ставимо дату схвалення
# - якщо немає дати доставки, то ставимо дату передачі перевізнику

olist_orders_cleaned = (
    olist_orders_cleaned
    .withColumn("order_approved_at", 
                when(col("order_approved_at").isNull(), col("order_purchase_timestamp"))
                .otherwise(col("order_approved_at")))
    .withColumn("order_delivered_carrier_date", 
                when(col("order_delivered_carrier_date").isNull(), col("order_approved_at"))
                .otherwise(col("order_delivered_carrier_date")))
    .withColumn("order_delivered_customer_date", 
                when(col("order_delivered_customer_date").isNull(), col("order_estimated_delivery_date"))
                .otherwise(col("order_delivered_customer_date")))
)



In [0]:
# Фільтрація аномалій (доставка раніше покупки)
olist_orders_cleaned = olist_orders_cleaned.filter(
    (col("order_delivered_customer_date") >= col("order_purchase_timestamp")) &
    (col("order_delivered_carrier_date") >= col("order_approved_at"))
)

# Приведення форматів дат до одного стандарту
date_columns = [
    "order_purchase_timestamp", "order_approved_at",
    "order_delivered_carrier_date", "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for c in date_columns:
    olist_orders_cleaned = olist_orders_cleaned.withColumn(c, to_timestamp(col(c), "yyyy-MM-dd HH:mm:ss"))



In [0]:
# Перевірка кількості пропусків після очищення
print("Пропуски після очищення:")
olist_orders_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in olist_orders_cleaned.columns]).show()

# Порівняння кількості рядків до і після очищення
print("Було рядків:", olist_orders_df.count())
print("Стало рядків:", olist_orders_cleaned.count())





Пропуски після очищення:
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|                0|                           0|                            0|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+

Було рядків: 99441
Стало рядків: 98082


In [0]:
# СТАНДАРТИЗАЦІЯ ТА ТРАНСФОРМАЦІЯ

from pyspark.sql.functions import col, year, month, dayofmonth, dayofweek, weekofyear, when

olist_orders_transformed = olist_orders_cleaned

# Розбиття дати покупки на складові компоненти
olist_orders_transformed = (
    olist_orders_transformed
    .withColumn("purchase_year", year(col("order_purchase_timestamp")))
    .withColumn("purchase_month", month(col("order_purchase_timestamp")))
    .withColumn("purchase_day", dayofmonth(col("order_purchase_timestamp")))
    .withColumn("purchase_weekday", dayofweek(col("order_purchase_timestamp")))
    .withColumn("purchase_weekofyear", weekofyear(col("order_purchase_timestamp")))
)

# Перетворення категоріальної змінної 'order_status' у числовий індекс
status_mapping = {
    "created": 0,
    "approved": 1,
    "processing": 2,
    "shipped": 3,
    "delivered": 4,
    "canceled": 5,
    "unavailable": 6,
    "invoiced": 7
}

olist_orders_transformed = olist_orders_transformed.withColumn("order_status_indexed", 
    when(col("order_status") == "created", 0)
    .when(col("order_status") == "approved", 1)
    .when(col("order_status") == "processing", 2)
    .when(col("order_status") == "shipped", 3)
    .when(col("order_status") == "delivered", 4)
    .when(col("order_status") == "canceled", 5)
    .when(col("order_status") == "unavailable", 6)
    .when(col("order_status") == "invoiced", 7)
    .otherwise(-1)
)



In [0]:
# Нормалізація часових різниць — обчислення тривалості доставки
from pyspark.sql.functions import datediff

olist_orders_transformed = olist_orders_transformed.withColumn(
    "delivery_time_days",
    datediff(col("order_delivered_customer_date"), col("order_purchase_timestamp"))
)

# Фільтрація нереалістичних показників
olist_orders_transformed = olist_orders_transformed.filter(
    (col("delivery_time_days") >= 0) & (col("delivery_time_days") <= 60)
)

display(olist_orders_transformed.limit(10))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchase_year,purchase_month,purchase_day,purchase_weekday,purchase_weekofyear,order_status_indexed,delivery_time_days
ccbabeb0b02433bd0fcbac46e70339f2,c77ee2d8ba1614a4d489a44166894938,delivered,2018-02-19T20:31:09.000Z,2018-02-21T06:15:25.000Z,2018-02-22T21:04:23.000Z,2018-03-09T22:22:25.000Z,2018-03-13T00:00:00.000Z,2018,2,19,2,8,4,18
c6bf92017bd40729c135b58b643f64c2,3d3c463710ea6e8dd9a63c1110eeb06b,delivered,2018-08-08T01:15:06.000Z,2018-08-09T07:44:53.000Z,2018-08-10T14:43:00.000Z,2018-08-15T00:18:43.000Z,2018-08-16T00:00:00.000Z,2018,8,8,4,32,4,7
ab87dc5a5f1856a10640d5f42e4c2fd9,538a4d02876412846b966a3c057395e5,delivered,2018-06-04T12:38:45.000Z,2018-06-04T12:50:47.000Z,2018-06-05T12:12:00.000Z,2018-06-11T20:39:49.000Z,2018-07-17T00:00:00.000Z,2018,6,4,2,23,4,7
06ff862a85c2402aa52dc9edf150bf30,0a978c825ff7d013133ddc7f77566172,delivered,2017-11-30T13:31:08.000Z,2017-12-01T11:30:56.000Z,2017-12-01T17:51:53.000Z,2017-12-28T22:27:57.000Z,2017-12-28T00:00:00.000Z,2017,11,30,5,48,4,28
f23155f5fa9b826631c5b8e038b38393,21a99191298d34fb6dd0b088e821591c,delivered,2017-09-20T12:19:12.000Z,2017-09-21T02:45:30.000Z,2017-09-26T19:22:57.000Z,2017-10-02T21:43:57.000Z,2017-10-13T00:00:00.000Z,2017,9,20,4,38,4,12
69fd81b0cd556f5da5000c1ed874ed19,6ad71323c11ba8a83737ccc3ea31fbc3,delivered,2017-09-25T10:53:40.000Z,2017-09-25T11:05:35.000Z,2017-09-25T18:06:56.000Z,2017-09-28T18:53:14.000Z,2017-10-19T00:00:00.000Z,2017,9,25,2,39,4,3
d40dd8018a5302969efb31bd21744cab,0470c47f1dd7a91d0f3b8a420589e0f7,delivered,2017-03-23T22:59:21.000Z,2017-03-23T23:10:30.000Z,2017-03-25T09:26:22.000Z,2017-04-06T16:33:45.000Z,2017-04-13T00:00:00.000Z,2017,3,23,5,12,4,14
42560dfc8d7863a190293678f01f6bbd,0f3a81be69f12da7e2979fd1833e923d,delivered,2017-10-22T01:39:47.000Z,2017-10-22T01:49:12.000Z,2017-10-23T14:07:54.000Z,2017-10-24T17:09:24.000Z,2017-11-06T00:00:00.000Z,2017,10,22,1,42,4,2
3f003568147c785083d014edfba38c48,72d90899884781ae2fc19e49cc102fc0,delivered,2018-06-18T16:58:12.000Z,2018-06-18T17:20:37.000Z,2018-06-26T15:27:00.000Z,2018-07-02T22:49:09.000Z,2018-07-17T00:00:00.000Z,2018,6,18,2,25,4,14
db192ddb0ea5a4d7a0506e7d7ec92343,c21e33cbee939409efff8c2ef959a275,delivered,2018-04-17T17:03:28.000Z,2018-04-17T17:15:17.000Z,2018-04-18T21:50:42.000Z,2018-04-23T20:11:24.000Z,2018-05-07T00:00:00.000Z,2018,4,17,3,16,4,6


In [0]:
# ЗБЕРЕЖЕННЯ РЕЗУЛЬТАТУ
delta_path = "/Volumes/workspace/default/olist_delta_dataset"

olist_orders_transformed.write.format("delta") \
    .mode("overwrite") \
    .save(delta_path)

print(f"Датасет успішно збережено у Delta Lake за шляхом: {delta_path}")

Датасет успішно збережено у Delta Lake за шляхом: /Volumes/workspace/default/olist_delta_dataset
